# Project: SendAudioFile (Server)

## 1. import

In [1]:
import socket
import _thread

## 2. 다중 클라이언트 연결을 위한 스레드 함수 선언

In [2]:
def threaded(conn, addr, group):
    print('Connect by',addr)
    while True:
        try:
            data = conn.recv(1024) # 파일명, 사이즈 수신
            if not data: # 소켓 연결 끊기면 연결 해제
                print('Disconnected by',addr)
                group.remove(conn)
                break

            file_info = data.decode()
            file_name, file_size = file_info.split("/")
            print('Receive File Path:',file_name)
            print('Receive File Size:',file_size)

            data = conn.recv(1024) # 파일 수신
            data_transferred = len(data)
            with open(file_name, "wb") as f:
                try:
                    while data:
                        f.write(data)
                        data = conn.recv(1024)
                        data_transferred += len(data)
                        
                        if data_transferred == int(file_size): # 파일 다 받으면 break
                            f.write(data)
                            break
                except Exception as ex:
                    print(ex)

            print("File is saved [byte:"+str(data_transferred)+"]")
            
            sendData = "완료\n"
            for c in group:
                if c is conn:
                    c.sendall(bytes(sendData,'UTF-8')) # 수신된 파일을 보낸 Client에게만 전송
                    print('Send Data : '+sendData,end="")
        except:
            # 클라이언트 소켓 강제 종료 시 (ex : 네트워크 변경)
            print('예외발생')
            print('Disconnected by',addr)
            group.remove(conn)
            break
    conn.close()

## 3. 소켓 생성

In [3]:
host = ''
port = 50000

server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
server_socket.bind((host, port))
server_socket.listen()
print("Listening")

Listening


## 4. 소켓 연결 대기

In [ ]:
group = []
while True:
    conn, addr = server_socket.accept()
    group.append(conn)
    _thread.start_new_thread(threaded, (conn, addr, group))
    
server_socket.close()
print('서버 종료')